In [1]:
!git clone -b optuna https://github.com/lionelsnaw/TuckER.git

In [2]:
!pip install -U kaleido

In [3]:
import torch

In [4]:
cd TuckER/

In [5]:
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
import numpy as np
import joblib

In [6]:
from main import Experiment, Data

In [7]:
SEED=42
def objective(trial):

    
#     data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
#     dtrain = lgb.Dataset(train_x, label=train_y)
#     dvalid = lgb.Dataset(valid_x, label=valid_y)

    params = {
        "num_iterations": 2,
        "batch_size": 128,
        "lr": trial.suggest_float("lr", 0.0001,  0.005),
        "dr": 1.0,
        'cuda': True,
        'input_dropout': trial.suggest_float("input_dropout", 0., 1.0),
        'hidden_dropout1': trial.suggest_float("hidden_dropout1", 0., 1.0),
        'hidden_dropout2': trial.suggest_float("hidden_dropout2", 0., 1.0),
        'label_smoothing': trial.suggest_float("label_smoothing", 0., 1.0),
        'edim': trial.suggest_int("edim", 20, 300),
        'rdim': trial.suggest_int("rdim", 20, 300),
    }
    
    dataset = 'WN18RR'
    data_dir = "data/%s/" % dataset
    torch.backends.cudnn.deterministic = True 
    seed = 20
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available:
        torch.cuda.manual_seed_all(seed) 
    d = Data(data_dir=data_dir, reverse=True)
    
    experiment = Experiment(num_iterations=params['num_iterations'], batch_size=params['batch_size'], learning_rate=params['lr'], 
                            decay_rate=params['dr'], ent_vec_dim=params['edim'], rel_vec_dim=params['rdim'], cuda=params['cuda'],
                            input_dropout=params['input_dropout'], hidden_dropout1=params['hidden_dropout1'], 
                            hidden_dropout2=params['hidden_dropout2'], label_smoothing=params['label_smoothing'])
    
    accuracy = experiment.train_and_eval(d=d, return_value=True)
    
    return accuracy

In [8]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)
study.optimize(objective, n_trials=5)

In [9]:
joblib.dump(study, "study.pkl")

In [10]:
fig = plot_optimization_history(study)
fig.update_layout(
    autosize=False,
    width=800,
    height=600)

fig.write_image("plot_optimization_history.png")
fig.show()

In [11]:
fig = plot_contour(study)
fig.update_layout(
    autosize=False,
    width=800,
    height=600)

fig.write_image("plot_contour.png")
fig.show()